# Creating Types

In [ ]:
import random
from deap import base, creator, tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE = 10

toolbox = base.Toolbox()
# toolbox.attr_float -> random.random()
toolbox.register("attr_float", random.random)
# toolbox.individual -> tools.initRepeat(creator.Individual, toolbox.attr_float, n=IND_SIZE)
toolbox.register("idividual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=IND_SIZE) # list of floats

## Permuation

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=10

toolbox = base.Toolbox()
# toolbox.indices -> random.sample(range(IND_SIZE), IND_SIZE)
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
# toolbox.individual -> tools.initIterate(creator.Individual, toolbox.indices)
# returns a complete creator.Individual composed of a permutation of [0, ..., IND_SIZE-1]
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.indices)

# One Max Problem
- population ov individuals consistent of integer vectors randomly filled with 0 and 1
- population evolves until one of its members contains only 1 and no 0 anymore

In [ ]:
import random

from deap import base, creator, tools

## Creator

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

## Toolbox

In [ ]:
toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, n=100)  # individual of 100 bits 
toolbox.register("population", tools.initRepeat, list, toolbox.individual) # population of individuals (no fixed size)

## The Evaluation Function

In [ ]:
def evalOneMax(individual):
    return sum(individual),  # count the number of 1s in the individual

## The Genetic Operators

In [ ]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

## Evolving the Population
- define an algorithm combining all the individual parts
- performing the evolution of our population unti the One Max problem is solved


In [ ]:
def main():
    pop = toolbox.population(n=300)  # create a population of 300 individuals

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # CXPB is the probability with which two individuals are crossed
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    
    # ------------
    # Performing the evolution
    # ------------

    # Extracting all the fitnesses of the individuals
    fits = [ind.fitness.values[0] for ind in pop]

    # evolve population until one of the individuals reaches fitness = 100
    g = 0 # generation counter

    # Begin the evolution
    while max(fits) < 100 and g < 1000:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)

        # -----------------
        # Evolution -> Selection, Crossover, Mutation
        # -----------------

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))    

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
                
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # replace the old population by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

# One Max Problem: Short Version

In [1]:
import array 
import random
import numpy

from deap import algorithms, base, creator, tools

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [3]:
def evalOneMax(individual):
    return sum(individual),

In [4]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [5]:
def main():
    random.seed(64)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, 
                                   stats=stats, halloffame=hof, verbose=True)

    return pop, log, hof